In [64]:
import pandas as pd
import numpy as np

In [65]:
train_app = pd.read_csv('../train/train_app.csv', encoding='utf-8')
test_app = pd.read_csv('../test/test_app.csv', encoding='utf-8')

In [66]:
train_app['month_id'] = train_app['month_id'].fillna(method='bfill')
train_app_3 = train_app[train_app['month_id']=='2020-03']

In [67]:
test_app['month_id'] = test_app['month_id'].fillna(method='bfill')

In [68]:
app_df = pd.concat([train_app_3, test_app], axis = 0)

In [69]:
app_df

,phone_no_m,busi_name,flow,month_id
11087,229a0f324de3c54b4cc4c82dafbf78ee5874fe491e4d9a...,NaN,125.060604,2020-03
11095,229a0f324de3c54b4cc4c82dafbf78ee5874fe491e4d9a...,QQ,7.247598,2020-03
11103,229a0f324de3c54b4cc4c82dafbf78ee5874fe491e4d9a...,旺信,0.008341,2020-03
11111,229a0f324de3c54b4cc4c82dafbf78ee5874fe491e4d9a...,微信,584.027821,2020-03
11130,229a0f324de3c54b4cc4c82dafbf78ee5874fe491e4d9a...,起点读书,5.342432,2020-03
...,...,...,...,...
129448,dde53a8d1df38164fc37e274ad7bafd9a85b515a68b7f8...,oppo应用商店,0.086297,2020-04
129449,e79ca3f725672a548ddb365746621e7f6fbd0b56798315...,NaN,0.000000,2020-04
129450,4dbbb1234ae86d3bb3e6ec490a3af338b3c1f977ae280e...,NaN,0.000000,2020-04
129451,82d9bf91308a6abdcee6cc251acaec7e139c65892fd37f...,NaN,0.000000,2020-04


In [70]:
# app_df['month_id'] = app_df['month_id'].fillna(method='bfill')

In [71]:
groups = app_df.groupby(by = ['phone_no_m'])

In [72]:
flow = groups['flow'].agg(np.sum).values
app_unique = groups['busi_name'].agg('nunique').values
flow_null = (flow<=10).astype(np.int64)
flow_min = groups['flow'].agg(np.min).values
flow_mean = groups['flow'].agg(np.mean).values
flow_max = groups['flow'].agg(np.max).values
months = groups['month_id'].agg(lambda x : len(x.value_counts())).values
# flow_per_month = flow/ months
flow_per_app = flow/(app_unique + 1)

In [73]:
app_list = app_df['busi_name'].value_counts()[app_df['busi_name'].value_counts().values >100].index.tolist()

In [74]:
len(app_list)

543

In [75]:
temp_df = pd.DataFrame()
for app in app_list:
    temp_df[app] = groups['busi_name'].agg(lambda x : (app in x.tolist())).values.astype(np.int64)

In [76]:
new_df = pd.DataFrame()
new_df['phone_no_m'] = groups.groups.keys()
new_df['flow'] = flow
new_df['flow_null'] = flow_null
new_df['flow_min'] = flow_min
new_df['flow_mean'] = flow_mean
new_df['flow_max'] = flow_max
new_df['flow_per_month'] = flow_per_month
new_df['flow_per_app'] = flow_per_app
new_df['app_unique'] = app_unique

In [77]:
new_df = pd.concat([new_df, temp_df],axis = 1)

In [78]:
new_df.to_csv('./app_submit.csv', encoding='utf-8', index = None)

In [79]:
new_df

,phone_no_m,flow,flow_null,flow_min,flow_mean,flow_max,flow_per_month,flow_per_app,app_unique,DNS,...,QiXiuZhiBo,铃声多多,寻医问药,Windows更新,咪咕音乐,ChinaDaily,AdView,中关村在线,美柚,疯狂猜图
0,00073ceecc0f7220a440580ac5dea410c90d14b6669458...,398.461888,0,0.000072,8.301289,203.986202,398.461888,9.266556,42,1,...,0,0,0,0,0,0,0,0,0,0
1,000f0f3100d815774e51f504a43e636dee84449da1c2c4...,3105.527115,0,0.001053,13.271483,1122.193042,3105.527115,15.605664,198,1,...,0,0,0,0,0,0,0,0,0,0
2,001285c54745996ac4705eca7713d73de7921edf4133a6...,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00281f7b748b9a009ab86a9da317856efdde8b1fc64388...,67.179778,0,0.000290,1.221451,30.974692,67.179778,1.291919,51,1,...,0,0,0,0,0,0,0,0,0,0
4,0034030646f2008d36431e00b133db30efc8b0c31757f3...,441.650370,0,0.000074,6.400730,207.131264,441.650370,6.794621,64,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6113,ffdbcf9285b71d560a7c79445700db2c363b066003a73d...,9406.169401,0,0.000644,47.505906,4190.440543,9406.169401,51.967787,180,1,...,0,0,0,0,0,0,0,0,0,0
6114,ffdf60d18dbbc25bb8c1697f7c97e8547517b2523d7001...,80.629025,0,0.000074,1.966562,47.127004,80.629025,2.015726,39,1,...,0,0,0,0,0,0,0,0,0,0
6115,ffe895705e90baa2490cfb0cc39031cf10cdd92facad6d...,10531.933698,0,0.000106,57.551550,4557.147507,10531.933698,61.952551,169,1,...,0,0,0,0,0,0,1,0,0,0
6116,ffe9b31003bfabebaad23e4e9eb593cf8566a906c3e4c0...,5532.343096,0,0.000074,34.362379,3773.408974,5532.343096,39.801029,138,1,...,0,0,0,0,0,0,0,0,0,0
